 ## Setup

In [13]:
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.v2 as T
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import random
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


 ## Paths

In [14]:
COCO_PATH = "../../data/coco/"  # change this
IMG_DIR_TRAIN = os.path.join(COCO_PATH, "images/train2017")
IMG_DIR_VAL = os.path.join(COCO_PATH, "images/val2017")
ANN_FILE_TRAIN = os.path.join(COCO_PATH, "annotations/person_keypoints_train2017.json")
ANN_FILE_VAL = os.path.join(COCO_PATH, "annotations/person_keypoints_val2017.json")

BATCH_SIZE = 32

In [15]:
class YOLOKeypointDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transform=None, grid_size=32, num_keypoints=17):
        super().__init__(img_folder, ann_file)
        self.coco = self.coco
        self.transform = transform
        self.grid_size = grid_size
        self.num_keypoints = num_keypoints
        self.image_size = 256  # resize all images

    def __getitem__(self, index):
        # Try to get the image and target, skip if file is missing
        try:
            img, target = super().__getitem__(index)
        except FileNotFoundError as e:
            img_info = self.coco.imgs[self.ids[index]]
            print(f"[Warning] Skipping missing file: {img_info.get('file_name', 'unknown')} (index {index})")
            # Try next index (wrap around if at end)
            return self.__getitem__((index + 1) % len(self))

        if self.transform:
            img = self.transform(img)

        target_tensor = torch.zeros((self.grid_size, self.grid_size, self.num_keypoints * 3))  # x, y, vis for each kp
        for ann in target:
            if ann["num_keypoints"] == 0:
                continue
            kps = torch.tensor(ann["keypoints"]).view(-1, 3)
            for i, (x, y, v) in enumerate(kps):
                if v > 0:
                    grid_x = int(x * self.grid_size / self.image_size)
                    grid_y = int(y * self.grid_size / self.image_size)
                    if 0 <= grid_x < self.grid_size and 0 <= grid_y < self.grid_size:
                        target_tensor[grid_y, grid_x, i*3 + 0] = x / self.image_size
                        target_tensor[grid_y, grid_x, i*3 + 1] = y / self.image_size
                        target_tensor[grid_y, grid_x, i*3 + 2] = 1.0  # visible
        return img, target_tensor

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.ConvertImageDtype(torch.float32)
])


train_dataset = YOLOKeypointDataset(IMG_DIR_TRAIN, ANN_FILE_TRAIN, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = YOLOKeypointDataset(IMG_DIR_VAL, ANN_FILE_VAL, transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

loading annotations into memory...
Done (t=13.43s)
creating index...
Done (t=13.43s)
creating index...
index created!
loading annotations into memory...
index created!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Done (t=0.47s)
creating index...
index created!


In [16]:
class YOLOKeypointNet(nn.Module):
    def __init__(self, num_keypoints=17, grid_size=32):
        super().__init__()
        self.backbone = torchvision.models.resnet18(weights="DEFAULT")
        self.backbone.fc = nn.Identity()
        # for param in self.backbone.parameters():
        #     param.requires_grad = False  # lock backbone

        self.head = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, num_keypoints * 3, kernel_size=1)  # (x, y, vis) per keypoint
        )
        self.grid_size = grid_size
        self.num_keypoints = num_keypoints

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), 512, 1, 1)
        x = torch.nn.functional.interpolate(x, size=(self.grid_size, self.grid_size), mode="bilinear")
        x = self.head(x)
        return x.permute(0, 2, 3, 1)  # [B, G, G, K*3]

In [17]:
def keypoint_loss(pred, target):
    mask = target[..., 2::3] > 0  # visibility mask
    pos_loss = ((pred[..., 0::3] - target[..., 0::3])**2 +
                (pred[..., 1::3] - target[..., 1::3])**2)
    conf_loss = ((pred[..., 2::3] - target[..., 2::3])**2)
    total_loss = (pos_loss * mask).sum() + conf_loss.sum()
    return total_loss / target.size(0)

In [18]:
model = YOLOKeypointNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# wandb.init(project="yolo-keypoint", name="resnet18-grid32", config={"epochs": 10, "grid_size": 32})

for epoch in range(10):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for imgs, targets in loop:
        imgs = imgs.to(device)
        targets = targets.to(device)

        preds = model(imgs)
        loss = keypoint_loss(preds, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        # wandb.log({"loss": loss.item()})

    print(f"Epoch {epoch+1}, avg loss: {total_loss / len(train_loader):.4f}")

Epoch 1:   0%|          | 0/3697 [00:00<?, ?it/s]

[Warning] Skipping missing file: 000000378670.jpg (index 77024)
[Warning] Skipping missing file: 000000378672.jpg (index 77025)
[Warning] Skipping missing file: 000000378684.jpg (index 77026)
[Warning] Skipping missing file: 000000378692.jpg (index 77027)
[Warning] Skipping missing file: 000000378700.jpg (index 77028)
[Warning] Skipping missing file: 000000378701.jpg (index 77029)
[Warning] Skipping missing file: 000000059281.jpg (index 12112)
[Warning] Skipping missing file: 000000216036.jpg (index 43860)
[Warning] Skipping missing file: 000000216038.jpg (index 43861)
[Warning] Skipping missing file: 000000216042.jpg (index 43862)
[Warning] Skipping missing file: 000000218558.jpg (index 44331)
[Warning] Skipping missing file: 000000218561.jpg (index 44332)
[Warning] Skipping missing file: 000000218569.jpg (index 44333)
[Warning] Skipping missing file: 000000218574.jpg (index 44334)
[Warning] Skipping missing file: 000000218578.jpg (index 44335)
[Warning] Skipping missing file: 0000001

KeyboardInterrupt: 

In [ ]:
# wandb.finish()

In [ ]:
def visualize_predictions(img, pred, threshold=0.5):
    img = img.permute(1, 2, 0).cpu().numpy()
    pred = pred.cpu().detach().numpy()

    plt.imshow(img)
    G = pred.shape[0]
    for y in range(G):
        for x in range(G):
            for i in range(17):
                conf = pred[y, x, i * 3 + 2]
                if conf > threshold:
                    px = pred[y, x, i * 3 + 0] * 256
                    py = pred[y, x, i * 3 + 1] * 256
                    plt.scatter([px], [py], c='r', s=10)
    plt.show()
